In [1]:
!pip install osmium pyrosm

  Using cached osmium-3.2.0-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.2 MB)
  Using cached pyrosm-0.6.1-cp39-cp39-linux_x86_64.whl
  Using cached pyrobuf-0.9.3-cp39-cp39-linux_x86_64.whl
  Using cached cykhash-2.0.0-cp39-cp39-linux_x86_64.whl
  Using cached python_rapidjson-1.6-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)


In [3]:
#https://pyrosm.readthedocs.io/en/latest/installation.html

In [4]:
from pyrosm.data import sources

# Print available source categories
sources.available.keys()

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


dict_keys(['africa', 'antarctica', 'asia', 'australia_oceania', 'central_america', 'europe', 'north_america', 'south_america', 'cities', 'subregions'])

In [2]:
from pyrosm import get_data, OSM

# Pyrosm comes with a couple of test datasets 
# that can be used straight away without
# downloading anything
fp = get_data("kenya")

# Initialize the OSM parser object
osm = OSM(fp)

# Read all drivable roads
# =======================
#drive_net = osm.get_network(network_type="driving")
#drive_net.plot()

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


Downloaded Protobuf data 'kenya-latest.osm.pbf' (173.61 MB) to:
'/tmp/pyrosm/kenya-latest.osm.pbf'


In [ ]:
print(sources.subregions.usa.available)

In [ ]:
from pyrosm import OSM
osm_path ='../california-latest.osm.pbf' # downloaded from geofabrik
osm = OSM(osm_path)

In [ ]:
drive_net = osm.get_network(network_type="driving")
drive_net.plot()

In [3]:
import osmium as osm
import pandas as pd

In [4]:
#https://stackoverflow.com/questions/45771809/how-to-extract-and-visualize-data-from-osm-file-in-python
#http://andrewgaidus.com/Convert_OSM_Data/
#https://github.com/agaidus/Converting_OSM_Data/blob/master/Convert_OSM_Data.ipynb
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []

    def tag_inventory(self, elem, elem_type):
        for tag in elem.tags:
            self.osm_data.append([elem_type, 
                                   elem.id, 
                                   elem.version,
                                   elem.visible,
                                   pd.Timestamp(elem.timestamp),
                                   elem.uid,
                                   elem.user,
                                   elem.changeset,
                                   len(elem.tags),
                                   tag.k, 
                                   tag.v])

    def node(self, n):
        self.tag_inventory(n, "node")

    def way(self, w):
        self.tag_inventory(w, "way")

    def relation(self, r):
        self.tag_inventory(r, "relation")

In [5]:
osmhandler = OSMHandler()

#FILE = "../california-latest.osm.pbf"
#FILE = "/tmp/pyrosm/socal-latest.osm.pbf"
FILE = "/tmp/pyrosm/kenya-latest.osm.pbf"

# scan the input file and fills the handler list accordingly
osmhandler.apply_file(FILE)

In [8]:
#https://techoverflow.net/2021/04/25/minimal-example-how-to-read-osm-pbf-file-using-python-osmium/
print(f'Number of nodes: {osmhandler.node_count}')
print(f'Number of way: {osmhandler.way_count}')
print(f'Number of relations: {osmhandler.relation_count}')

Number of nodes: <bound method OSMHandler.node of <__main__.OSMHandler object at 0x7f7188a45f40>>
Number of way: <bound method OSMHandler.way of <__main__.OSMHandler object at 0x7f7188a45f40>>
Number of relations: <bound method OSMHandler.relation of <__main__.OSMHandler object at 0x7f7188a45f40>>


In [12]:
# transform the list into a pandas DataFrame
data_colnames = ['type', 'id', 'version', 'visible', 'ts', 'uid',
                 'user', 'chgset', 'ntags', 'tagkey', 'tagvalue']
df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)
#df_osm = tag_genome.sort_values(by=['type', 'id', 'ts'])

In [13]:
df_osm.head()

,type,id,version,visible,ts,uid,user,chgset,ntags,tagkey,tagvalue
0,node,25579897,1,True,2007-01-31 21:15:11+00:00,0,,0,1,created_by,JOSM
1,node,25579912,1,True,2007-01-31 21:15:17+00:00,0,,0,1,created_by,JOSM
2,node,25579913,1,True,2007-01-31 21:15:17+00:00,0,,0,1,created_by,JOSM
3,node,25579921,1,True,2007-01-31 21:15:18+00:00,0,,0,1,created_by,JOSM
4,node,27565105,34,True,2022-02-28 10:20:48+00:00,0,,0,89,admin_level,2


In [14]:
len(df_osm)

4927341

In [ ]:
# In terminal:
# wget -c http://download.geofabrik.de/north-america/us/california-latest.osm.pbf

# Tutorial: https://www.linuxbabe.com/linux-server/openstreetmap-tile-server-ubuntu-16-04

In [ ]:
# https://stackoverflow.com/questions/45771809/how-to-extract-and-visualize-data-from-osm-file-in-python
